In [39]:
#######################################################
# Created-Date: 04/11/2016                            #
# Last Modified: 04/27/2016                           #
# Created by: Himanshu Bharara , Shravan Kumar        #
#                                                     #
# Objective: build a logistic regression model        #
#            on the training and test data for        #
#            different scenarios mentioned in ppt     #
#                                                     #
# Instruction before running:                         #
# 1. Please change the file path of directory         # 
# 2. change ranges based on the number of lyrics      # 
#    you have in the directory                        #  
# 3. Please follow similar nomenclature as used in the# 
#    code                                             #
# 4. Please use the array list of only the years      #
#    under consideration and comment the rest         #
# 5. Please use the scenario definition to assign 0's #
#    to the relevant subset of the super set          #
#######################################################



import pandas as pd
import matplotlib.pyplot as plt
import pandas.io.data
from statsmodels import regression
import math
import statsmodels.api as sm
import datetime
import numpy as np

dhome = "/Users/HimanshuBharara/Documents/CU-Sem2/IEORE4571/Projects/Datasets/songs"
year = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
#year = ['2030','2020']
## Adding 1's to 2020 Dataset
filepath = dhome+year[0]+".csv"
df=pd.read_csv(filepath)
grades = []
for row in df['ranks']:
    if  row <= 20:
        grades.append(1)
    else:
        grades.append(0)
df['grades'] = grades

## Adding 0's to 2030 Dataset
#filepath = dhome+year[11]+".csv"
#df1=pd.read_csv(filepath)
#grades = []
#for row in df1['ranks']:
#    if  row <= 103:
#        grades.append(0)
#    else:
#        grades.append(1)
#df1['grades'] = grades

#df = df.append(df1)

## Adding 0's to Remaining Dataset
for i in range(1,10):
    filepath = dhome+year[i]+".csv"
    mds = pd.read_csv(filepath,header=0)
    grades = []
    for row in mds['ranks']:
        if  row <= 20:
            grades.append(1)
        else:
            grades.append(0)
    mds['grades'] = grades
    df = df.append(mds)

print(df.shape)
#print(list(df.columns.values))

## Drop Duplicates
od = df.sort(['songs','artists'], ascending=[1,0])
ot=od.drop_duplicates(['songs','artists'], keep='first')
print(ot.shape)
ot.pop('ranks')
ot.pop('ids')
ot.pop('artists')
ot.pop('songs')
#ot['grades'].value_counts()
## Print the summary statistics of the modelling data set to an external file to analyse before preceding for modelling
pd.DataFrame(ot.describe()).to_csv("/Users/HimanshuBharara/Documents/CU-Sem2/IEORE4571/Projects/SummaryStatistics.csv")

(1000, 23)
(909, 23)


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:54: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [40]:
from sklearn.cross_validation import train_test_split
df_train,df_test = train_test_split(ot,random_state=0)

print(df_train.shape)
print(df_test.shape)
print(list(df_train.columns.values))
print(df_train['grades'].value_counts())
print(df_test['grades'].value_counts())
# Print the summary statistics of the training data set to an external file to analyse before preceding for modelling
pd.DataFrame(df_train.describe()).to_csv("/Users/HimanshuBharara/Documents/CU-Sem2/IEORE4571/Projects/TrainingSummaryStatistics.csv")

(681, 19)
(228, 19)
0    551
1    130
Name: grades, dtype: int64
0    173
1     55
Name: grades, dtype: int64


In [41]:
## Logistic Regression Without PCA
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
y = df_train['grades']
df_train.pop('grades')
model_lr = LogisticRegression()
model_lr.fit(df_train, y)  
print(model_lr.score(df_train, y))

0.809104258443


In [43]:
## Predicting Logistic Regression on Testing DataSet
z = df_test['grades']
df_test.pop('grades')
fit = model_lr.predict(df_test)

In [44]:
## Summary Statistics for Classifier Algorithm
print(metrics.classification_report(fit, z))
print(metrics.accuracy_score(fit, z))     
print(metrics.confusion_matrix(fit, z))

             precision    recall  f1-score   support

          0       1.00      0.76      0.86       228
          1       0.00      0.00      0.00         0

avg / total       1.00      0.76      0.86       228

0.758771929825
[[173  55]
 [  0   0]]


//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [47]:
## Calculate Area under the ROC courve
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(z, fit,pos_label=1) # recall my labels are 4,5 and 6
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

Area under the ROC curve : 0.500000


In [48]:
# Evaluating the model using cross-validation(using 10-Fold)
md_Y = ot['grades']
ot.pop('grades')
md_X = ot

In [49]:
## Calculate cross validation metrics for understanding modelling accuracy
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(LogisticRegression(), md_X, md_Y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.79347826  0.79347826  0.79347826  0.79347826  0.79120879  0.8         0.8
  0.8         0.8         0.8       ]
0.796512183469
